In [5]:
import numpy as np
import random
import json
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


import torch
import torch.nn as nn
import nltk
nltk.download('punkt')

from torch.utils.data import Dataset, DataLoader

import numpy as np
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shop\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [49]:
# ANN Model

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

In [50]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [51]:


def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [52]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)



242 patterns
68 tags: ['Hypersomnia', 'anxious_false', 'anxious_true', 'appetite_less', 'appetite_okay', 'appetite_over', 'crying_false', 'crying_true', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-33', 'fact-34', 'fact-35', 'fact-36', 'fact-37', 'fact-38', 'fact-39', 'fact-40', 'fact-41', 'fact-42', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'feel_bad', 'feel_good', 'good bye', 'greetings', 'harm_false', 'harm_true', 'hopeless_false', 'hopeless_true', 'insomnia', 'joke', 'loneliness_false', 'loneliness_true', 'mentally_tired_false', 'mentally_tired_true', 'proper_sleep', 'relations_bad', 'relations_good', 'smoking_false', 'smoking_true']
286 unique stemmed words: ["'m", "'s", ',', '4', 'a', 'abl', 'about', 'ach', 'activ', 'addict', 'adequ

In [53]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)



In [54]:
# Hyper-parameters 
num_epochs = 200
batch_size = 32
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)



286 68


In [55]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples



In [56]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)



In [57]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track accuracy
        _, predicted_labels = torch.max(outputs, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_samples

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2%}')


Epoch [1/200], Loss: 4.2590, Accuracy: 0.41%
Epoch [2/200], Loss: 4.2505, Accuracy: 0.41%
Epoch [3/200], Loss: 4.2302, Accuracy: 2.48%
Epoch [4/200], Loss: 4.2258, Accuracy: 3.72%
Epoch [5/200], Loss: 4.2106, Accuracy: 3.72%
Epoch [6/200], Loss: 4.2005, Accuracy: 4.13%
Epoch [7/200], Loss: 4.1937, Accuracy: 4.13%
Epoch [8/200], Loss: 4.1708, Accuracy: 4.55%
Epoch [9/200], Loss: 4.1628, Accuracy: 4.55%
Epoch [10/200], Loss: 4.1521, Accuracy: 4.55%
Epoch [11/200], Loss: 4.1307, Accuracy: 4.55%
Epoch [12/200], Loss: 4.1022, Accuracy: 4.96%
Epoch [13/200], Loss: 4.0832, Accuracy: 4.96%
Epoch [14/200], Loss: 4.0578, Accuracy: 4.55%
Epoch [15/200], Loss: 4.0276, Accuracy: 4.55%
Epoch [16/200], Loss: 3.9953, Accuracy: 4.55%
Epoch [17/200], Loss: 3.9696, Accuracy: 4.55%
Epoch [18/200], Loss: 3.9283, Accuracy: 4.55%
Epoch [19/200], Loss: 3.9010, Accuracy: 4.96%
Epoch [20/200], Loss: 3.8608, Accuracy: 5.37%
Epoch [21/200], Loss: 3.8098, Accuracy: 7.02%
Epoch [22/200], Loss: 3.7711, Accuracy: 6.6

In [58]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track accuracy
        _, predicted_labels = torch.max(outputs, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_samples

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2%}')

print(f'Final loss: {average_loss:.4f}, Final accuracy: {accuracy:.2%}')


Epoch [100/200], Loss: 0.0386, Accuracy: 100.00%
Epoch [200/200], Loss: 0.0101, Accuracy: 100.00%
Final loss: 0.0101, Final accuracy: 100.00%


In [59]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


## Chat with chatbot for testing

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Medical ChatBot"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
